## Import libraries

In [1]:
! pip install PyDictionary
! python -m spacy download en_core_web_md

     |████████████████████████████████| 115 kB 1.0 MB/s 
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=d5452a70a0e73b2837c8bf02d7db8ace299863d16fd207e378a0b7c5309a22b2
  Stored in directory: /root/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
  Created wheel for goslate: filename=goslate-1.5.1-py3-none-any.whl size=11549 sha256=e71a6f5e9848569f68550bfd0c4aff5f0370bb9daa509975c6ec7e99a410e4f4
  Stored in directory: /root/.cache/pip/wheels/16/5c/86/f561bb944dc9f853f9502427bde8fe557a6fafe146d4a432f2
Successfully built bs4 goslate
     |████████████████████████████████| 50.8 MB 24.0 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.3.1-py3-none-any.whl size=50916640 sha256=cd358392db83c00953f2c24fbc62e085129b35517bc2e25686a943df24a1f360
  Stored in directory: /tmp/pip-ephem-wheel-cache-49paqm9y/wheels/43/1d/c1/a0af68d0648debf57f875e9dda56bbac35cfc27bfa187ffc46
Successfully built en-core-web-md
✔ Download and 

In [2]:
import warnings
warnings.filterwarnings("ignore")

import re
import nltk
import spacy
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import en_core_web_md
from nltk.corpus import wordnet
from PyDictionary import PyDictionary

## Load source datasets

In [3]:
train_df = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
train_df.drop(['url_legal','license','standard_error'], inplace=True, axis=1)
train_df.set_index("id", inplace=True)
print(f"train_df: {train_df.shape}\n")
train_df.head()

train_df: (2834, 2)



,excerpt,target
id,,
c12129c31,When the young people returned to the ballroom...,-0.340259
85aa80a4c,"All through dinner time, Mrs. Fayre was somewh...",-0.315372
b69ac6792,"As Roger had predicted, the snow departed as q...",-0.580118
dd1000b26,And outside before the palace a great garden w...,-1.054013
37c1b32fb,Once upon a time there were Three Bears who li...,0.247197


## Set percentage level 

**(number of words to be replaced for augmentation)**

In [4]:
percent = 20

## Word Similarity Check

**(aim is to return the most similar word to the given input word)**

In [5]:
def synalter_Noun_Verb(word, a1, POS):
    
    max_temp = -1
    flag = 0
    
    for i in a1:
        try:
            w1 = wordnet.synset(word + '.' + POS + '.01') 
            w2 = wordnet.synset(i + '.' + POS + '.01')

            if(max_temp < w1.wup_similarity(w2)):
                max_temp = w1.wup_similarity(w2)
                temp_name = i
                flag = 1
        except:
            f=0
            
    if flag == 0:
        max1 = -1.
        nlp = en_core_web_md.load()
        
        for i in a1:
            j=i.replace(' ', '')
            tokens = nlp(u'' + j)
            token_main = nlp(u'' + word)
            
            for token1 in token_main:
                if max1<float(token1.similarity(tokens)):
                    max1 = token1.similarity(tokens)
                    value = i
                    
        max1 = -1.
        return value
    
    else:
        return temp_name

## Text Replacement

1. Count the total number of Unique words and make a dictionary
2. Filter out unique words and eliminate 2 letters word & numbers
3. Make a list of Noun and Verb using spacy from the above-filtered list
4. Random selection from Noun and Verb along with above mention percentage
5. Loop all the randomly selected words
   * create a word_str variable to store a word.
   * Finding out all the synonyms using the thesaurus and making a list.
   * Checking If a selected word is Noun or Verb and than passing to `synalter_Noun_Verb(word, al, POS)`
   
   
6. Replacing the word with the new most similar word

In [6]:
def text_replacement(text):

    synonyms = [] 
    antonyms = []

    output_text = text
    print(f"Original Text: \n\n{text}\n")
    print("="*90)

    words = text.split()
    counts = {}

    for word in words:
        if word not in counts:
            counts[word] = 0

        counts[word] += 1

    one_word = []

    for key, value in counts.items():
        if value == 1 and key.isalpha() and len(key)>2:
            one_word.append(key)

    noun = []
    verb = []

    nlp = en_core_web_md.load()
    doc = nlp(u''+' '.join(one_word))

    for token in doc:
        if  token.pos_ == 'VERB':
            verb.append(token.text)

        if  token.pos_ == 'NOUN':
            noun.append(token.text)

    all_main =verb + noun
    len_all = len(noun)+len(verb)
    final_value = int(len_all * percent /100)

    random.seed(0)
    dictionary=PyDictionary()
    temp = random.sample(range(0, len_all), final_value)

    for i in tqdm(temp):
        word_str = all_main[i]
        a1 = dictionary.synonym(word_str)
        
        if a1 is None:
            continue
        
        else:
            if i < len(verb):
                change_word=synalter_Noun_Verb(word_str, a1, 'v')
                search_word = re.search(r'\b('+word_str+r')\b', output_text)
                Loc = search_word.start()
                output_text = output_text[:int(Loc)] + '[' + change_word + ']' + output_text[int(Loc) + len(word_str):]

            else:
                change_word=synalter_Noun_Verb(word_str, a1, 'n')
                search_word = re.search(r'\b('+word_str+r')\b', output_text)
                Loc = search_word.start()
                output_text = output_text[:int(Loc)] + '[' + change_word + ']' + output_text[int(Loc) + len(word_str):]
    
    print("="*90)
    print(f'\nAugmented Text:\n\n{output_text}\n')

In [7]:
%%time
text_replacement(train_df.iloc[0]['excerpt'])

Original Text: 

When the young people returned to the ballroom, it presented a decidedly changed appearance. Instead of an interior scene, it was a winter landscape.
The floor was covered with snow-white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. The numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. Also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches.
At each end of the room, on the wall, hung a beautiful bear-skin rug.
These rugs were for prizes, one for the girls and one for the boys. And this was the game.
The girls were gathered at one end of the room and the boys at the other, and one end was called the North Pole, and the other the South Pole. Each player was given a small flag which they were to plant on reaching the Pole.
This would have been an easy matter, but each traveller was obliged to wear s

100%|██████████| 7/7 [00:39<00:00,  5.57s/it]



Augmented Text:

When the young people returned to the ballroom, it presented a decidedly [transformed] appearance. Instead of an interior scene, it was a winter landscape.
The floor was covered with snow-white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. The numerous palms and evergreens that had decorated the room, were powdered with [semolina] and strewn with tufts of cotton, like snow. Also diamond [fallout] had been lightly sprinkled on them, and glittering crystal icicles hung from the branches.
At each end of the room, on the wall, hung a beautiful bear-skin rug.
These [broadloom] were for prizes, one for the girls and one for the [herb]. And this was the game.
The girls were gathered at one end of the room and the boys at the other, and one end was called the North Pole, and the other the South Pole. Each player was [granted] a small flag which they were to plant on reaching the Pole.
This would have been an easy matter, but each t

In [8]:
%%time
text_replacement(train_df.iloc[100]['excerpt'])

Original Text: 

The priest, smiling at this speech, answered: "Being such as I am, I have no desire and no wishes. Glad as I am to hear your kind intentions, there is nothing that I can ask you to do for me. You need feel no anxiety on my account. As long as I live, when the winter comes, you shall be welcome here." The badger, on hearing this, could not conceal its admiration of the depth of the old man's benevolence; but having so much to be grateful for, it felt hurt at not being able to requite it. As this subject was often renewed between them, the priest at last, touched by the goodness of the badger's heart, said: "Since I have shaven my head, renounced the world, and forsaken the pleasures of this life, I have no desire to gratify, yet I own I should like to possess three riyos in gold. Food and raiment, I receive by the favor of the villagers, so I take no heed for those things.



 67%|██████▋   | 4/6 [00:08<00:03,  1.81s/it]

could has no Synonyms in the API


 83%|████████▎ | 5/6 [00:09<00:01,  1.47s/it]

should has no Synonyms in the API


100%|██████████| 6/6 [00:16<00:00,  2.80s/it]



Augmented Text:

The priest, smiling at this speech, answered: "Being such as I am, I have no desire and no wishes. Glad as I am to hear your kind intentions, there is nothing that I [keep] ask you to do for me. You need feel no anxiety on my account. As long as I live, when the winter comes, you shall be welcome here." The badger, on hearing this, could not conceal its [philhellenism] of the depth of the old man's benevolence; but having so much to be grateful for, it felt hurt at not being able to [give] it. As this subject was often renewed between them, the priest at last, touched by the goodness of the badger's heart, said: "Since I have shaven my head, renounced the world, and [abandon] the pleasures of this life, I have no desire to gratify, yet I own I should like to possess three riyos in gold. Food and raiment, I receive by the favor of the villagers, so I take no heed for those things.

CPU times: user 19.2 s, sys: 972 ms, total: 20.2 s
Wall time: 23.3 s


In [9]:
%%time
text_replacement(train_df.iloc[1000]['excerpt'])

Original Text: 

Most mornings, Hare would take some seeds from Hyena's granary. Then he went to his field and pretended to work. He would light a fire and roast the seeds. He did nothing, except eat roasted seeds. At harvest time, Hare went and stole his friend's ripe maize. He carried it to his own granary. Hyena complained about the stolen crops. Hare answered, "Maybe the neighbors stole your maize. They are suffering from famine because they are lazy." Hyena thought about this problem for a long time. He came up with an idea. He collected gum from the trees. He put that sticky gum around the whole field. The thief would get stuck in the gum. During the night, Hare went to Hyena's field to steal more maize. Before entering the field, he looked around to see if it was safe. Hare did not see the gum that Hyena had put around the field. Hare stood in the gum and was stuck. He started shouting for help. Hyena heard the shouts and ran out to catch the thief. He could not believe his eyes

100%|██████████| 6/6 [00:29<00:00,  4.94s/it]



Augmented Text:

Most mornings, Hare would take some seeds from Hyena's granary. Then he went to his field and [assumed] to work. He would light a fire and roast the seeds. He did nothing, except eat roasted seeds. At [collect] time, Hare went and stole his friend's ripe maize. He carried it to his own granary. Hyena complained about the stolen crops. Hare answered, "Maybe the neighbors stole your maize. They are suffering from [disaster] because they are lazy." Hyena thought about this problem for a long time. He [get] up with an idea. He collected gum from the trees. He put that sticky gum around the whole field. The thief would get stuck in the gum. During the night, Hare went to Hyena's field to steal more maize. Before entering the field, he looked around to see if it was safe. Hare did not see the gum that Hyena had put around the field. Hare stood in the gum and was stuck. He [begin] [cheering] for help. Hyena heard the shouts and ran out to catch the thief. He could not believ